In [1]:
#%pip install xgboost
#%pip install wordcloud
#%pip install gensim

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

In [3]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [4]:
df = train[['facts', 'first_party_winner']]
df_target = df[['first_party_winner']]
df_nlp = df[['facts']]
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)

In [5]:
corpus = df_nlp1['facts']
# print(corpus.str.cat(sep=" ")) # 인덱스의 요소들 서로 잇기
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")

In [6]:
# 문장 깨끗하게 만드는 함수

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list # 역토큰화, TF-IDF를 사용하기 위해서
    text = " ".join(lst_text)
    return text

In [7]:
# 문장 깨끗하게 만드는 함수 사용
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

# y값 포함해서 하나의 프레임으로 만들기
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')
df_nlp2 = df_nlp2.drop(columns='facts')
df_nlp2

,facts_clean,first_party_winner
0,june 27 1962 phil st amant candidate public of...,1
1,ramon nelson riding bike suffered lethal blow ...,0
2,alabama state court convicted billy joe magwoo...,1
3,victor linkletter convicted state court eviden...,0
4,april 24 1953 selma alabama intruder broke apa...,1
...,...,...
2473,congress amended clean air act energy policy a...,1
2474,alliance bond fund inc investment fund purchas...,1
2475,1992 district court sentenced manuel peguero 2...,0
2476,march 8 1996 enrico st cyr lawful permanent re...,0


In [8]:
#TFIDF 적용, 문서들의 유사도를 구하는 작업

vectorizeTF = TfidfVectorizer(stop_words='english')
count_matrix_tf = vectorizeTF.fit_transform(df_nlp2['facts_clean'])
data_final = count_matrix_tf.toarray()
data_final = pd.DataFrame(data=data_final, columns=vectorizeTF.get_feature_names_out())
data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
data_final

,011119,0125,02,036539,04,041352,041581,0479,05,0511287,...,zone,zoned,zoneofinterests,zoning,zubik,zuni,zurich,zurko,zurkos,first_party_winner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [9]:
# 카운터 벡터라이즈는 사용중단
# vectorize=CountVectorizer()
# count_matrix = vectorize.fit_transform(df_nlp2['facts_clean'])
# count_array = count_matrix.toarray()
# data_final = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# data_final

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=0)

In [11]:
# LDA 학습, 단어의 의미구조 파악
# 30초정도 걸림

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
lda_data = lda.fit_transform(X_train)

In [12]:
# LDA 학습 결과 보기
terms = vectorizeTF.get_feature_names_out()

def get_topics(components, feature_names, n=5): # n: n개 출력
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda.components_,terms)

# 토픽과 토픽에 대한 기여도

Topic 1: [('sale', 1.64), ('abortion', 1.55), ('minor', 1.07), ('parent', 0.97), ('notification', 0.81)]
Topic 2: [('applicable', 1.32), ('motor', 1.21), ('initially', 1.01), ('arbitrator', 0.88), ('investment', 0.83)]
Topic 3: [('film', 0.73), ('fsa', 0.66), ('obscene', 0.65), ('eberhart', 0.47), ('athlete', 0.45)]
Topic 4: [('registration', 0.95), ('rodgers', 0.79), ('beaudreaux', 0.72), ('rippo', 0.69), ('discon', 0.61)]
Topic 5: [('fsia', 1.08), ('idaho', 1.08), ('foreign', 0.82), ('forest', 0.66), ('central', 0.56)]
Topic 6: [('tax', 18.33), ('court', 14.52), ('action', 13.93), ('board', 12.61), ('plan', 12.38)]
Topic 7: [('lovings', 0.57), ('walton', 0.46), ('chapel', 0.38), ('inability', 0.38), ('fdca', 0.36)]
Topic 8: [('rico', 2.82), ('racketeer', 1.1), ('corrupt', 1.07), ('influenced', 1.05), ('districting', 0.76)]
Topic 9: [('clinton', 0.77), ('busing', 0.74), ('stamp', 0.56), ('tolling', 0.48), ('reopen', 0.46)]
Topic 10: [('redistricting', 2.28), ('plan', 1.5), ('voting', 

In [13]:
# XG부스트 모델 적용하기

model = XGBClassifier()
x = pd.DataFrame(data=lda_data)
model.fit(x, y_train)
x_test = pd.DataFrame(data=lda.transform(X_test))
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]

In [14]:
# 평가

from sklearn.metrics import f1_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(y_test, y_pred)

Accuracy: 60.08%


0.7206020696142993

# 테스트csv 적용

In [15]:
dfTest = pd.DataFrame(test['facts'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
dfTest

,facts,facts_clean,first_party_winner
0,The 1984 Bail Reform Act allowed the federal c...,1984 bail reform act allowed federal court det...,0
1,Lexecon Inc. was a defendant in a class action...,lexecon inc defendant class action lawsuit 28 ...,0
2,"In 2002 and 2003, Fox Television Stations broa...",2002 2003 fox television station broadcast bil...,0
3,During his trial for armed robbery of a federa...,trial armed robbery federally insured saving l...,0
4,"In 1993, a magistrate judge issued a warrant a...",1993 magistrate judge issued warrant authorizi...,0
...,...,...,...
1235,According to Executive Order No. 12807 signed ...,according executive order 12807 signed preside...,0
1236,Section 109(a) of the Clean Air Act (CAA) requ...,section 109a clean air act caa requires enviro...,0
1237,Linda Matteo and John Madigan created a plan f...,linda matteo john madigan created plan utilizi...,0
1238,"In 1972, the North Carolina Board of Agricultu...",1972 north carolina board agriculture adopted ...,0


In [16]:
count_matrix_test = vectorizeTF.transform(dfTest['facts_clean'])
count_array_test = count_matrix_test.toarray()
data_final_test = pd.DataFrame(data=count_array_test,columns = vectorizeTF.get_feature_names_out())
data_final_test = pd.concat([data_final_test,dfTest["first_party_winner"]],axis=1,join='inner')
data_final_test

,011119,0125,02,036539,04,041352,041581,0479,05,0511287,...,zone,zoned,zoneofinterests,zoning,zubik,zuni,zurich,zurko,zurkos,first_party_winner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [17]:
X_test = data_final_test.drop(columns=['first_party_winner'])
y_test = data_final_test['first_party_winner']

In [18]:
x_lda_test = pd.DataFrame(data=lda.fit_transform(X_test))
y_pred_test = model.predict(x_lda_test)
predictionsTest = [round(value) for value in y_pred_test]
predcsv = pd.DataFrame(predictionsTest,columns=['first_party_winner'])

In [19]:
submit = pd.read_csv('C:/data/project/sample_submission.csv')
submit['first_party_winner'] = predcsv
submit.to_csv('./sample_submission.csv', index=False)
print('Done')

Done


In [20]:
predcsv.value_counts()

first_party_winner
0                     918
1                     322
Name: count, dtype: int64